In [65]:
import numpy as np
import matplotlib.pyplot as plt
def fonction_g_discrete(choix_angle,v):
    if v==0:
        print("ici")
        x=0
        y=0
    else: 
        if choix_angle==1:
            x=1
            y=0
        if choix_angle==2:
            x=-1
            y=0
        if choix_angle==3:
            x=0
            y=1
        if choix_angle==4:
            x=0
            y=-1
    return([x,y])



In [66]:
import random
import numpy as np
import pandas as pd

def overall():
    xcible=5
    ycible=5
    lim_x=[0,19]
    lim_y=[0,19]
    destination=[xcible,ycible]
    Liste_realisations=[]
    table_cout=np.random.multivariate_normal(size=20,mean=[-5]*20,cov=np.eye(20))
    goal_x=5
    goal_y=5
    for J in range(1000):
        T=np.random.choice([20,50,100])
        Table_nom=[J]*T
        Recompenses=[]
        Actions=[]
        Nouvel_etat=[]
        Positions_t=[]
        x_si=np.random.choice(range(20))
        y_si=np.random.choice(range(20))
        situation_initiale=(x_si,y_si)
        for j in range(T):
            vitesse=np.random.choice([0,1,2,3])
            if situation_initiale[0]==goal_x and situation_initiale[1]==goal_y:
                recompense=1
            else:
                recompense=-1
            if vitesse!=0:
                choix_angle=random.choice([1,2,3,4])
                changement_pos=fonction_g_discrete(choix_angle=choix_angle,v=vitesse)
                situation_nouvelle=np.add(changement_pos,np.array(situation_initiale))
                if situation_nouvelle[0]<lim_x[0] or situation_nouvelle[0]>lim_x[1]:
                    recompense=+-20*vitesse
                    situation_nouvelle=situation_initiale
                elif situation_nouvelle[1]<lim_y[0] or situation_nouvelle[1]>lim_y[1]:
                    recompense=+-20*vitesse
                    situation_nouvelle=situation_initiale
                elif situation_nouvelle[0]==goal_x and situation_nouvelle[1]==goal_y:
                    recompense=1
                else:
                    recompense=+table_cout[situation_nouvelle[0],situation_nouvelle[1]]/vitesse
            else:
                situation_nouvelle=situation_initiale
                choix_angle=random.choice([0,1,2,3,4])
            Recompenses.append(recompense)
            Actions.append((vitesse,choix_angle))
            Positions_t.append(tuple(situation_initiale))
            Nouvel_etat.append(tuple(situation_nouvelle))
            situation_initiale=tuple(situation_nouvelle)
        dico=pd.DataFrame({"recompense":Recompenses,"actions":Actions,"etat_t":Positions_t,"etat_suivant":Nouvel_etat,"Indice_simul":Table_nom})
        Liste_realisations.append(dico)
    return(pd.concat(Liste_realisations))

In [67]:
R=overall()

In [68]:
R

,recompense,actions,etat_t,etat_suivant,Indice_simul
0,-3.915653,"(1, 3)","(10, 14)","(10, 15)",0
1,-1.000000,"(0, 1)","(10, 15)","(10, 15)",0
2,-1.000000,"(0, 2)","(10, 15)","(10, 15)",0
3,-1.804585,"(3, 3)","(10, 15)","(10, 16)",0
4,-2.260527,"(2, 3)","(10, 16)","(10, 17)",0
...,...,...,...,...,...
15,-1.343104,"(3, 4)","(5, 11)","(5, 10)",999
16,-3.023846,"(2, 1)","(5, 10)","(6, 10)",999
17,-5.030496,"(1, 3)","(6, 10)","(6, 11)",999
18,-0.785156,"(3, 1)","(6, 11)","(7, 11)",999


In [69]:
def q_learning(situations, eta, gamma):
    # Initialisation du dictionnaire Q avec des valeurs nulles
    liste_simulations = np.unique(situations["id_simulation"].tolist())
    Q={}
    for simulation in liste_simulations:
            sous_base = situations[situations["id_simulation"] == simulation]
            for row in sous_base.itertuples():
                etat_k = row.s_t
                action_k = row.a_t
                cle = (etat_k, action_k)
                Q[cle]=0
    
    
    threshold = 1e-3

    convergence = False
    while not convergence:
        Q_ant = dict(Q)  # Sauvegarde des valeurs de Q de l'itération précédente
        
        for simulation in liste_simulations:
            sous_base = situations[situations["id_simulation"] == simulation]
            for row in sous_base.itertuples():
                etat_k =row.s_t
                action_k = row.a_t
                cle = (etat_k, action_k)
                recompense = row.r_t
                situation_t_plus = row.s_t_p_1


                list_cle = [key for key in list(Q_ant.keys()) if key[0] == situation_t_plus]

                if len(list_cle) == 0:
                     Q_max = 0
                else:
                    Q_max = np.max([Q_ant[cle] for cle in list_cle])

                # Calcul de la valeur de Q en utilisant la formule de mise à jour de Q-learning
                # Q_max = np.max([Q_ant.get((situation_t_plus, int(a))) for a in actions["id"].tolist()])
                Q[cle] = (1 - eta) * Q_ant[cle] + eta * (recompense + gamma * Q_max)
        
        # Vérification de la convergence
        dist = sum(np.absolute(np.subtract(list(Q.values()),list(Q_ant.values()))))
        # convergence = all(abs(Q[k] - Q_ant[k]) < 1e-6 for k in Q.keys())
        if dist < threshold:
            convergence = True
    
    return Q



In [70]:
eta=0.7
gamma=0.6

In [71]:
R

,recompense,actions,etat_t,etat_suivant,Indice_simul
0,-3.915653,"(1, 3)","(10, 14)","(10, 15)",0
1,-1.000000,"(0, 1)","(10, 15)","(10, 15)",0
2,-1.000000,"(0, 2)","(10, 15)","(10, 15)",0
3,-1.804585,"(3, 3)","(10, 15)","(10, 16)",0
4,-2.260527,"(2, 3)","(10, 16)","(10, 17)",0
...,...,...,...,...,...
15,-1.343104,"(3, 4)","(5, 11)","(5, 10)",999
16,-3.023846,"(2, 1)","(5, 10)","(6, 10)",999
17,-5.030496,"(1, 3)","(6, 10)","(6, 11)",999
18,-0.785156,"(3, 1)","(6, 11)","(7, 11)",999


In [72]:
R = R.rename(columns={'recompense': 'r_t', 'actions': 'a_t',"Indice_simul":"id_simulation","etat_t":"s_t","etat_suivant":"s_t_p_1"})
R

,r_t,a_t,s_t,s_t_p_1,id_simulation
0,-3.915653,"(1, 3)","(10, 14)","(10, 15)",0
1,-1.000000,"(0, 1)","(10, 15)","(10, 15)",0
2,-1.000000,"(0, 2)","(10, 15)","(10, 15)",0
3,-1.804585,"(3, 3)","(10, 15)","(10, 16)",0
4,-2.260527,"(2, 3)","(10, 16)","(10, 17)",0
...,...,...,...,...,...
15,-1.343104,"(3, 4)","(5, 11)","(5, 10)",999
16,-3.023846,"(2, 1)","(5, 10)","(6, 10)",999
17,-5.030496,"(1, 3)","(6, 10)","(6, 11)",999
18,-0.785156,"(3, 1)","(6, 11)","(7, 11)",999


In [73]:
QL = q_learning(eta=eta,gamma=gamma,situations=R)

In [ ]:
QL

{((13, 7), (1, 2)): -7.58486911428707,
 ((12, 7), (0, 1)): -2.499998680012399,
 ((12, 7), (1, 2)): -7.666337500384302,
 ((11, 7), (3, 4)): -3.6067595630992466,
 ((11, 6), (2, 1)): -4.033963684244147,
 ((12, 6), (3, 1)): -3.167661804469284,
 ((13, 6), (3, 1)): -3.3655215467586204,
 ((14, 6), (3, 3)): -2.954864036678858,
 ((14, 7), (3, 3)): -2.9235036782342636,
 ((14, 8), (3, 1)): -3.099668001621541,
 ((15, 8), (3, 1)): -2.6218112532457254,
 ((16, 8), (0, 1)): -2.499998680012399,
 ((16, 8), (3, 3)): -3.5741380135443093,
 ((16, 9), (1, 4)): -4.865436399712378,
 ((16, 8), (3, 1)): -2.8504150289643615,
 ((17, 8), (3, 3)): -2.861304874438415,
 ((17, 9), (2, 2)): -4.611207680310264,
 ((16, 9), (0, 4)): -2.499998680012399,
 ((16, 9), (2, 4)): -3.182717539862388,
 ((16, 8), (1, 2)): -6.2990066448398245,
 ((15, 8), (1, 2)): -5.770513674677992,
 ((14, 8), (0, 1)): -2.499998680012399,
 ((14, 8), (1, 2)): -6.666569889142061,
 ((13, 8), (3, 2)): -3.0235572292434485,
 ((12, 8), (1, 1)): -6.6665698891

In [ ]:
Etats=[]
best_action=[]
best_reward=[]
for cle in QL.keys():
    Liste_actions=[]
    Liste_recompense=[]
    Etats.append(cle[0])
    for cle2,valeur_recompense in QL.items():
        if cle[0]==cle2[0]:
            Liste_actions.append(cle2[1])
            Liste_recompense.append(valeur_recompense)
    indice_meilleur=np.argmax(Liste_recompense)
    MR_action=Liste_actions[indice_meilleur]
    MR_recompense=Liste_recompense[indice_meilleur]
    best_action.append(MR_action)
    best_reward.append(MR_recompense)
Best_action_per_state=pd.DataFrame({"etats":Etats,"Meilleure_action":best_action,"Meilleure_recompense":best_reward})

In [ ]:
Best_action_per_state

,etats,Meilleure_action,Meilleure_recompense
0,"(13, 7)","(3, 3)",-3.222189
1,"(12, 7)","(0, 1)",-2.499999
2,"(12, 7)","(0, 1)",-2.499999
3,"(11, 7)","(0, 0)",-2.499999
4,"(11, 6)","(0, 0)",-2.499999
...,...,...,...
1486,"(11, 7)","(0, 0)",-2.499999
1487,"(11, 8)","(0, 3)",-2.499999
1488,"(11, 8)","(0, 3)",-2.499999
1489,"(11, 7)","(0, 0)",-2.499999
